In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn import datasets

iris=datasets.load_iris()
print(iris.data)
print(iris.target)
print(iris.data.shape)
print(iris.target.shape)

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6.9 3.1 4.

1. pytorch 기본 소스로 모델작성
2. 클래스로 모델 작성

In [46]:
x_train=torch.FloatTensor(iris.data)
y_train = torch.LongTensor(iris.target)

y_one_hot=torch.zeros(150,3)
y_one_hot.scatter_(1,y_train.unsqueeze(1),1)
print(y_train[145:])
print(y_one_hot[145:])


tensor([2, 2, 2, 2, 2])
tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.]])


In [47]:
w=torch.zeros((4,3),requires_grad=True)
b=torch.zeros((1,3),requires_grad=True)

optimizer=optim.SGD([w,b], lr=0.01)


In [48]:
for epoch in range(4001):
  # y 
  y_hat=F.softmax(x_train.matmul(w)+b, dim=1)
  cost=(y_one_hot * -torch.log(y_hat)).sum(dim=1).mean() 

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()


  if epoch % 400 ==0:
    print('epoch: {}, cost:{:.4f}'.format(epoch,cost.item())) 


epoch: 0, cost:1.0986
epoch: 400, cost:0.4770
epoch: 800, cost:0.3860
epoch: 1200, cost:0.3341
epoch: 1600, cost:0.2977
epoch: 2000, cost:0.2703
epoch: 2400, cost:0.2488
epoch: 2800, cost:0.2314
epoch: 3200, cost:0.2171
epoch: 3600, cost:0.2051
epoch: 4000, cost:0.1948


In [49]:
# 만들어진 결과의 정확도
pred=F.softmax(x_train.matmul(w)+b, dim=1).argmax(dim=1)
accuracy=(pred==y_train).sum()/len(pred)*100
print(accuracy)
# 정확도 98% 150개중 3개 정도


tensor(98.)


In [50]:
class SoftmaxClassModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear=nn.Linear(4,3)

  def forward(self,x):
    return self.linear(x)

In [51]:
# 모델생성
model=SoftmaxClassModel()
optimzer=optim.SGD(model.parameters(),lr=0.1)


In [55]:

for epoch in range(5001):
  y_hat=model(x_train)
  cost=F.cross_entropy(y_hat,y_train)
  
  optimzer.zero_grad()
  cost.backward()
  optimzer.step()
  
  if epoch%500 == 0:
    accuracy=(y_hat.argmax(dim=1)==y_train).sum()/len(y_hat)
    print('epoch: {}, cost:{:.5f}, accuracy:{:.5f}'.format(epoch,cost.item(),accuracy))

epoch: 0, cost:0.05906, accuracy:0.98000
epoch: 500, cost:0.05862, accuracy:0.98000
epoch: 1000, cost:0.05820, accuracy:0.98000
epoch: 1500, cost:0.05780, accuracy:0.98000
epoch: 2000, cost:0.05743, accuracy:0.98000
epoch: 2500, cost:0.05707, accuracy:0.98000
epoch: 3000, cost:0.05673, accuracy:0.98000
epoch: 3500, cost:0.05641, accuracy:0.98000
epoch: 4000, cost:0.05610, accuracy:0.98000
epoch: 4500, cost:0.05581, accuracy:0.98000
epoch: 5000, cost:0.05553, accuracy:0.98000


In [59]:
# 정확도
predict=model(x_train).argmax(dim=1)
print(predict)
(predict==y_train).sum()/len(predict)*100

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])


tensor(98.)